In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-12-06"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
22493,2024-12-06,Itália Série A2,16:30,Cividale,Avellino Basket,1.31,3.05,152.5,1.85,1.81,-8.5,2.00,1.69,xlUYPWr9,0.763359,0.327869,0.540541,0.552486,0.091228,174.052,77.330057,0.444293,2.4,1.341641,0.559017,131.67,2.204,0.765885,0.347498,12.0,186.926,60.628878,0.324347,0.6,1.341641,2.236068,170.24,2.630,0.850441,0.323362,-39.0,77,64,1.71,2.66,137.458,0.000,0.564388,0.015456,0.118809,4.01,0.802,0.386534,0.552958,0.9,0.347042,0.00,0.000,inf,0.000000,0.0,0.000000
22494,2024-12-06,França Pro B,16:30,ASA,Chalons-Reims,1.21,3.74,167.5,1.85,1.81,-9.5,1.94,1.73,xtEAzqSb,0.826446,0.267380,0.540541,0.552486,0.093826,135.786,50.780707,0.373976,0.6,1.341641,2.236068,102.96,1.766,0.746378,0.422638,-6.0,173.208,79.664076,0.459933,1.2,1.643168,1.369306,119.93,2.100,0.845665,0.402698,-33.0,78,67,1.32,1.79,110.922,192.888,0.722820,0.015456,0.080922,-2.32,-0.464,-0.452586,0.704577,0.5,-0.204577,0.66,0.132,20.757576,0.439725,0.5,0.060275
22495,2024-12-06,França Lfb Feminina,16:00,Flammes Carolo F,Landerneau Bretagne F,1.21,3.74,144.5,1.83,1.83,-10.5,1.95,1.72,d0qPg8Ga,0.826446,0.267380,0.546448,0.546448,0.093826,243.024,136.759249,0.562740,3.0,0.000000,0.000000,224.11,2.862,1.408304,0.492070,40.0,200.822,163.279148,0.813054,1.2,1.643168,1.369306,105.35,3.110,2.115715,0.680294,-30.0,73,49,3.07,2.15,183.358,305.522,0.722820,0.000000,0.088629,4.58,0.916,0.229258,0.547594,0.7,0.152406,4.76,0.952,2.878151,0.286509,0.6,0.313491
22496,2024-12-06,Grécia Liga De Elite,14:30,Koroivos,Lefkadas,1.47,2.43,147.5,1.81,1.85,-5.5,1.93,1.74,EuML9LrH,0.680272,0.411523,0.552486,0.540541,0.091795,99.800,20.867281,0.209091,1.2,1.643168,1.369306,131.24,1.542,0.243660,0.158015,3.0,160.272,44.347057,0.276699,1.2,1.643168,1.369306,135.34,2.164,0.649484,0.300131,-4.0,68,67,1.93,2.02,102.798,160.272,0.348114,0.015456,0.073215,-1.99,-0.398,-1.180905,0.686135,0.6,-0.086135,-1.87,-0.374,-3.823529,0.430002,0.2,-0.230002
22497,2024-12-06,Hungria Nacional,14:00,Budapesti Honved Se,Alba Fehervar,4.20,1.17,170.5,1.80,1.86,8.5,2.01,1.68,8I1excNB,0.238095,0.854701,0.555556,0.537634,0.092796,327.350,295.402775,0.902407,1.2,1.643168,1.369306,837.50,4.814,4.460586,0.926586,-46.0,130.436,37.241005,0.285512,1.2,1.643168,1.369306,92.43,1.396,0.476477,0.341316,12.0,67,79,12.50,1.17,278.732,138.310,0.797964,0.023184,0.126474,-2.94,-0.588,-5.442177,0.471249,0.4,-0.071249,-5.00,-1.000,-0.170000,0.616523,0.2,-0.416523
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22602,2024-12-06,China Wcba Feminina,08:30,Shaanxi Tianze F,Liaoning F,1.95,1.72,152.5,1.83,1.83,1.5,1.83,1.83,0xGBYVLb,0.512821,0.581395,0.546448,0.546448,0.094216,124.938,71.158164,0.569548,1.8,1.643168,0.912871,101.00,1.680,0.924905,0.550539,39.0,147.938,61.516944,0.415829,1.2,1.643168,1.369306,233.32,1.982,0.871763,0.439840,-22.0,100,76,1.01,3.07,142.854,143.776,0.088629,0.000000,0.000000,1.39,0.278,3.417266,0.434762,0.3,-0.134762,-1.30,-0.260,-2.769231,0.506330,0.5,-0.006330
22603,2024-12-06,China Wcba Feminina,08:30,Jiangsu Phoenix F,Hebei F,1.01,9.70,159.5,1.83,1.83,-26.5,1.95,1.72,jRngGl6U,0.990099,0.103093,0.546448,

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
22501,08:00,Israel Liga Leumit,Maccabi Raanana,Ramat Hasharon,1.56,Back Home
22502,08:00,Israel Liga Leumit,Maccabi Rishon,Elitzur Yavne,1.64,Back Home
22537,12:00,Turquia Tb2L,Gemlik,Istanbul BB,2.09,Back Home
22566,16:30,Europa Euroliga,Virtus Bologna,Crvena zvezda,1.81,Back Home
22585,20:30,Eua Ncaa,Niagara,Siena,1.73,Back Home
22602,08:30,China Wcba Feminina,Shaanxi Tianze F,Liaoning F,1.95,Back Home
